In [ ]:
#student: Yuqing Qiao
#date: 08/03/2024

# Fine-Tuning LLMs

In this exercise, you will fine-tune the [Flan-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) model for enhanced dialogue summarization. You will first explore a full fine-tuning approach and evaluate the results with ROUGE metrics. Then you will perform Parameter-Efficient Fine-Tuning (PEFT), evaluate the resulting model and see that the benefits of PEFT outweigh the slightly-lower performance metrics.

## 1. Set up Dependencies and Load Dataset and LLM

In [3]:
!pip install datasets evaluate rouge_score peft -q --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
gcsfs 2024.6.1 requires fsspec==2024.6.1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
from datasets import load_dataset

In [26]:
dataset = load_dataset('knkarthick/dialogsum')

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Load the pre-trained [Flan-T5 model](https://huggingface.co/docs/transformers/model_doc/flan-t5) and its tokenizer from HuggingFace. Notice that you will be using the [small version](https://huggingface.co/google/flan-t5-base) of Flan-T5. Setting `torch_dtype=torch.bfloat16` specifies the data type to be used by this model, which can reduce GPU memory usage since `bfloat16` uses half as much memory per number compared to `float32`, the default precision for most models.

In [6]:
model_name = 'google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

## 2. Test the Model with Zero-Shot Inferencing

Test the model with zero-shot inference.

In [5]:
index = 42
dash_line = '-' * 100

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
inputs = tokenizer(prompt, return_tensors='pt')
output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
original_model_summary = tokenizer.decode(output, skip_special_tokens=True)

print(dash_line)
print(f'INPUT PROMPT:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{original_model_summary}\n')

----------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: I don't know how to adjust my life. Would you give me a piece of advice?
#Person2#: You look a bit pale, don't you?
#Person1#: Yes, I can't sleep well every night.
#Person2#: You should get plenty of sleep.
#Person1#: I drink a lot of wine.
#Person2#: If I were you, I wouldn't drink too much.
#Person1#: I often feel so tired.
#Person2#: You better do some exercise every morning.
#Person1#: I sometimes find the shadow of death in front of me.
#Person2#: Why do you worry about your future? You're very young, and you'll make great contribution to the world. I hope you take my advice.
----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# wants to adjust #Person1#'s life and #Person2# suggests #Person1# be positive and stay healthy.
-------------------------------------------------------

You can see that the model struggles to summarize the dialogue compared to the baseline summary, and simply repeats the first sentence from the dialogue.

## 3. Perform Full Fine-Tuning

### 3.1 Preprocess the Dataset

You need to convert the dialog-summary (prompt-response) pairs into explicit instructions for the LLM. Prepend an instruction to the start of the dialog with `Summarize the following conversation.`, and to the start of the summary with `Summary:` as follows:

Training prompt (dialogue):
```
Summarize the following conversation.
Alice: This is her part of the conversation.
Bob: This is his part of the conversation.    
Summary:
```

Training response (summary):
```
Both Alice and Bob participated in the conversation.
```

**Exercise**: Write a function to tokenize a batch of examples from the dialogue dataset. The function should concatentate the dialogues with the predefined prompt, tokenize them along with their summaries, and define the tokenized summaries as the labels.

In [27]:
from datasets import concatenate_datasets
import numpy as np

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]
# take 85 percentile of max length for better utilization
max_source_length = int(np.percentile(input_lenghts, 85))
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# take 90 percentile of max length for better utilization
max_target_length = int(np.percentile(target_lenghts, 90))
print(f"Max target length: {max_target_length}")

def tokenize(examples):
    # format prompt input
    inputs = ["Summarize the following conversation.\n" + item + "Summary:\n" for item in examples["dialogue"]]

    model_inputs = tokenizer(inputs, max_length=max_source_length, padding='max_length', truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, padding="max_length", truncation=True)

    # found solution online to correct calc loss with max_length padding, ignore padding in loss.
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Map:   0%|          | 0/13960 [00:00<?, ? examples/s]

Max source length: 324


Map:   0%|          | 0/13960 [00:00<?, ? examples/s]

Max target length: 62


In [29]:
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [8]:
# a quick check tokenizer
tokenizer.decode(tokenized_dataset['train'][0]['labels'])

"Summary: Mr. Smith's getting a check-up, and Doctor Hawkins advises him to have one every year. Hawkins'll give some information about their classes and medications to help Mr. Smith quit smoking.</s>"

### 3.2 Fine-Tune the Model

**Exercise**: Utilize the Hugging Face Trainer API for training the model on the preprocessed dataset. Define the training arguments, a data collator, and create a `Seq2SeqTrainer` instance. Train the model for one epoch.

In [15]:
### WRITE YOUR CODE HERE
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    num_train_epochs=1,
    predict_with_generate=True,
    logging_steps=50
)


data_collator = DataCollatorForSeq2Seq(tokenizer, model=original_model)

trainer = Seq2SeqTrainer(
    model = original_model,
    args = training_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['validation'],
    data_collator = data_collator,
    tokenizer = tokenizer
)



Training a fully fine-tuned version of the model should take about 10 minutes on a Google Colab GPU machine.

In [16]:
trainer.train()

Step,Training Loss,Validation Loss
50,1.515800,1.309082
100,1.501600,1.298340
150,1.477800,1.292969
200,1.473000,1.289551
250,1.453900,1.287109
300,1.443400,1.285645
350,1.444100,1.285645


TrainOutput(global_step=390, training_loss=1.472636217948718, metrics={'train_runtime': 307.3024, 'train_samples_per_second': 40.546, 'train_steps_per_second': 1.269, 'total_flos': 5399204730716160.0, 'train_loss': 1.472636217948718, 'epoch': 1.0})

Save the model to a local folder:

In [17]:
model_path = './flan-t5-base-dialogsum-checkpoint'

original_model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('./flan-t5-base-dialogsum-checkpoint/tokenizer_config.json',
 './flan-t5-base-dialogsum-checkpoint/special_tokens_map.json',
 './flan-t5-base-dialogsum-checkpoint/spiece.model',
 './flan-t5-base-dialogsum-checkpoint/added_tokens.json',
 './flan-t5-base-dialogsum-checkpoint/tokenizer.json')

Create an instance of the `AutoModelForSeq2SeqLM` class for the instruct model:

In [18]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained('./flan-t5-base-dialogsum-checkpoint',
                                                       torch_dtype=torch.bfloat16)


Reload the original Flan-T5-base model:

In [19]:
original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)

### 3.3 Evaluate the Model Qualitatively (Human Evaluation)

**Exercise**: Make inferences for the same example as in Section 2, using the original model and the fully fine-tuned model.

In [13]:
### WRITE YOUR CODE HERE
index = 42
dash_line = '-' * 100

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
inputs = tokenizer(prompt, return_tensors='pt')
output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
original_model_summary = tokenizer.decode(output, skip_special_tokens=True)

output = instruct_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
instruct_model_summary = tokenizer.decode(output, skip_special_tokens=True)

print(dash_line)
print(f'INPUT PROMPT:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{original_model_summary}\n')
print(dash_line)
print(f'MODEL GENERATION - INSTRUCT:\n{instruct_model_summary}\n')


----------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: I don't know how to adjust my life. Would you give me a piece of advice?
#Person2#: You look a bit pale, don't you?
#Person1#: Yes, I can't sleep well every night.
#Person2#: You should get plenty of sleep.
#Person1#: I drink a lot of wine.
#Person2#: If I were you, I wouldn't drink too much.
#Person1#: I often feel so tired.
#Person2#: You better do some exercise every morning.
#Person1#: I sometimes find the shadow of death in front of me.
#Person2#: Why do you worry about your future? You're very young, and you'll make great contribution to the world. I hope you take my advice.
----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# wants to adjust #Person1#'s life and #Person2# suggests #Person1# be positive and stay healthy.
-------------------------------------------------------

The fine-tuned model is able to create a much better summary of the dialogue compared to the original model.

### 3.4 Evaluate the Model Quantitatively (with ROUGE Metric)

The [ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While not perfect, it does indicate the overall increase in summarization effectiveness that we have accomplished by fine-tuning.

In [44]:
rouge = evaluate.load('rouge')

**Exercise**: Generate the outputs for a sample of the test set with the fine-tuned model (use only the first 10 dialogues and summaries to save time).

In [20]:
### WRITE YOUR CODE HERE

human_baseline_summaries = dataset['test'][0:10]['summary']
# output using the first 10 dialogues
dialogues = dataset['test'][0:10]['dialogue']

prompt_template = "Summarize the following conversation.\n{dialogue}\nSummary:\n"
prompts = [prompt_template.format(dialogue=dialogue) for dialogue in dialogues]

inputs = tokenizer(
    prompts,
    max_length=512,
    padding = 'max_length',
    truncation=True,
    return_tensors='pt'
)

assert inputs['input_ids'].shape[0] == 10

# instruct model summaries
instruct_outputs = instruct_model.generate(inputs['input_ids'], max_new_tokens=50)
instruct_model_summaries = tokenizer.batch_decode(instruct_outputs, skip_special_tokens=True)

# original model summaries
original_outputs = original_model.generate(inputs['input_ids'], max_new_tokens=50)
original_model_summaries = tokenizer.batch_decode(original_outputs, skip_special_tokens=True)

In [18]:
instruct_model_summaries[0]

'Summary: #Person1# needs to take a dictation from #Person2# to all employees by this afternoon. #Person1# will take a dictation from #Person2# to all employees by'

Evaluate the models computing ROUGE metrics:

In [17]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)]
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)]
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.22420174087325248, 'rouge2': 0.09872463768115944, 'rougeL': 0.2089424816139932, 'rougeLsum': 0.21333914728682168}
INSTRUCT MODEL:
{'rouge1': 0.3275091799271346, 'rouge2': 0.07437680252489523, 'rougeL': 0.260491697625684, 'rougeLsum': 0.2629241661738963}


The results show substantial improvement in all ROUGE metrics:

In [19]:
print("Absolute percentage improvement of the instruct model over the original model:")

for key in instruct_model_results:
    improvement = instruct_model_results[key] - original_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the instruct model over the original model:
rouge1: 10.33%
rouge2: -2.43%
rougeL: 5.15%
rougeLsum: 4.96%


## 4. Perform Parameter Efficient Fine-Tuning (PEFT)

Now, let's perform **Parameter Efficient Fine-Tuning (PEFT)** instead of "full fine-tuning" as you did above. PEFT is a form of instruction fine-tuning that is much more efficient than full fine-tuning, with comparable evaluation results as you will see soon.

One of the most popular PEFT methods is **Low-Rank Adaptation (LoRA)**, which  introduces low-rank matrices to adapt the LLM with minimal additional parameters. In most cases, when someone says PEFT, they typically mean LoRA.  After fine-tuning for a specific task with LoRA, the result is that the original LLM remains unchanged and a newly-trained "LoRA adapter" emerges. This LoRA adapter is much smaller than the original LLM - on the order of a single-digit % of the original LLM size (MBs vs GBs).  

At inference time, the LoRA adapter is reunited and combined with its original LLM to serve the inference request. The benefit is that many LoRA adapters can re-use the original LLM which reduces overall memory requirements when serving multiple tasks and use cases.

### 4.1 Setup the LoRA model for Fine-Tuning

You first need to define the configuration of the LoRA model. Have a look at the configuration below. The key configuration element to adjust is the rank (`r`) of the adapter, which influences its capacity and complexity. Experiment with various ranks, such as 8, 16, or 32, and see how they affect the results.

In [21]:
from peft import LoraConfig, TaskType, get_peft_model

### WRITE YOUR CODE HERE
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=32,
    lora_alpha=32,
    lora_dropout=0.1
)

# create new peft model with lora rank
def apply_lora_rank(model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base'), r=32):
    lora_config = LoraConfig(
        task_type=TaskType.SEQ_2_SEQ_LM,
        r=r,
        lora_alpha=32,
        lora_dropout=0.1
    )

    peft_model = get_peft_model(model, lora_config)
    return peft_model

# r=8 , 16 lora model is evaluated at the bottom cells

peft_model8 = apply_lora_rank(r=8)
peft_model8.print_trainable_parameters()


peft_model16 = apply_lora_rank(r=16)
peft_model16.print_trainable_parameters()


trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3561
trainable params: 1,769,472 || all params: 249,347,328 || trainable%: 0.7096


Add LoRA adapter layers/parameters to the original LLM to be trained:

In [22]:
peft_model = get_peft_model(original_model, lora_config)

The number of trainable model parameters in the LoRA model is:

In [23]:
peft_model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 251,116,800 || trainable%: 1.4093


### 4.2 Train the LoRA Adapter

**Exercise**: Define training arguments and create a `Seq2SeqTrainer` instance for the LoRA model. Use a higher learning rate than full fine-tuning (e.g., `1e-3`).

In [30]:

### WRITE YOUR CODE HERE
# Define the training arguments
peft_training_args = Seq2SeqTrainingArguments(
    auto_find_batch_size=True,
    output_dir="./results",
    eval_strategy="steps",
    # eval_steps = 100,
    learning_rate=1e-3,  # Higher learning rate for LoRA
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    num_train_epochs=1,
    predict_with_generate=True,
    save_strategy="no",
    # logging_dir='./logs'
    logging_steps=100,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=peft_model)

# r =32 lora
peft_trainer  = Seq2SeqTrainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    )
# r = 8 lora
peft_trainer8 = Seq2SeqTrainer(
    model=peft_model8,
    args=peft_training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)
# r= 16 lora
peft_trainer16 = Seq2SeqTrainer(
    model=peft_model16,
    args=peft_training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer
)

Train the PEFT adapter. Training should take about 6 minutes on a Google Colab GPU machine.

In [31]:
peft_trainer.train()

Step,Training Loss,Validation Loss
100,1.375500,1.154785
200,1.259800,1.118408
300,1.223000,1.099609


TrainOutput(global_step=390, training_loss=1.2708133012820513, metrics={'train_runtime': 285.5541, 'train_samples_per_second': 43.634, 'train_steps_per_second': 1.366, 'total_flos': 5484925881630720.0, 'train_loss': 1.2708133012820513, 'epoch': 1.0})

Save the model to a local folder:

In [35]:
peft_model.save_pretrained('./flan-t5-base-dialogsum-lora')

In [32]:
peft_trainer8.train()
peft_trainer8.save_model('./flan-t5-base-dialogsum-lora8')
del peft_trainer8, peft_model8

Step,Training Loss,Validation Loss
100,1.372800,1.157562
200,1.260000,1.122595
300,1.221300,1.100703


TrainOutput(global_step=390, training_loss=1.2695173605894432, metrics={'train_runtime': 552.8188, 'train_samples_per_second': 22.539, 'train_steps_per_second': 0.705, 'total_flos': 5442065306173440.0, 'train_loss': 1.2695173605894432, 'epoch': 1.0})

In [33]:
peft_trainer16.train()
peft_trainer16.save_model('./flan-t5-base-dialogsum-lora16')
del peft_trainer16, peft_model16

Step,Training Loss,Validation Loss
100,1.230100,1.102664
200,1.200100,1.084763
300,1.178900,1.071169


TrainOutput(global_step=390, training_loss=1.1985457004644933, metrics={'train_runtime': 552.5906, 'train_samples_per_second': 22.548, 'train_steps_per_second': 0.706, 'total_flos': 5442065306173440.0, 'train_loss': 1.1985457004644933, 'epoch': 1.0})

Load the PEFT model:

In [36]:
peft_model = AutoModelForSeq2SeqLM.from_pretrained('./flan-t5-base-dialogsum-lora')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')


Reload the original Flan-T5-base model:

In [28]:
original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)

### 4.3 Evaluate the Model Qualitatively (Human Evaluation)

**Exercise**: Make inferences for the same example as in Sections 2 and 3, using the original model, the fully fine-tuned model and the PEFT model.

In [20]:
### WRITE YOUR CODE HERE
index = 42
dash_line = '-' * 100

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
inputs = tokenizer(prompt, return_tensors='pt')
output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
original_model_summary = tokenizer.decode(output, skip_special_tokens=True)

peft_model_output = peft_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
peft_model_summary = tokenizer.decode(peft_model_output, skip_special_tokens=True)

print(dash_line)
print(f'INPUT PROMPT:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{original_model_summary}\n')
print(dash_line)
print(f'MODEL GENERATION - PEFT:\n{peft_model_summary}\n')

----------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: I don't know how to adjust my life. Would you give me a piece of advice?
#Person2#: You look a bit pale, don't you?
#Person1#: Yes, I can't sleep well every night.
#Person2#: You should get plenty of sleep.
#Person1#: I drink a lot of wine.
#Person2#: If I were you, I wouldn't drink too much.
#Person1#: I often feel so tired.
#Person2#: You better do some exercise every morning.
#Person1#: I sometimes find the shadow of death in front of me.
#Person2#: Why do you worry about your future? You're very young, and you'll make great contribution to the world. I hope you take my advice.
----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# wants to adjust #Person1#'s life and #Person2# suggests #Person1# be positive and stay healthy.
-------------------------------------------------------

### 4.4 Evaluate the Model Quantitatively (with ROUGE Metric)

**Exercise**: Generate the outputs for a sample of the test set with the PEFT model (use only the first 10 dialogues and summaries to save time).

In [38]:
### WRITE YOUR CODE HERE
human_baseline_summaries = dataset['test'][0:10]['summary']
# output using the first 10 dialogues
dialogues = dataset['test'][0:10]['dialogue']

prompt_template = "Summarize the following conversation.\n{dialogue}\nSummary:\n"
prompts = [prompt_template.format(dialogue=dialogue) for dialogue in dialogues]

inputs = tokenizer(
    prompts,
    max_length=512,
    truncation=True,
    padding=True,
    return_tensors='pt'
)

assert inputs['input_ids'].shape[0] == 10

# instruct model summaries
peft_outputs = peft_model.generate(inputs['input_ids'], max_new_tokens=50)
peft_model_summaries = tokenizer.batch_decode(peft_outputs, skip_special_tokens=True)

# original model summaries
original_outputs = original_model.generate(inputs['input_ids'], max_new_tokens=50)
original_model_summaries = tokenizer.batch_decode(original_outputs, skip_special_tokens=True)


Compute ROUGE score for this subset of the data.

In [32]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.22420174087325248, 'rouge2': 0.09872463768115944, 'rougeL': 0.2089424816139932, 'rougeLsum': 0.21333914728682168}
INSTRUCT MODEL:
{'rouge1': 0.34365952324619925, 'rouge2': 0.08409931608682723, 'rougeL': 0.26088436510201424, 'rougeLsum': 0.260550773220114}
PEFT MODEL:
{'rouge1': 0.38903449499676734, 'rouge2': 0.1435584221155926, 'rougeL': 0.3076366713741615, 'rougeLsum': 0.3109300424297905}


Notice, that PEFT model results are not too bad, while the training process was much easier!

Calculate the improvement of PEFT over the original model:

In [33]:
print("Absolute percentage improvement of the PEFT model over the original model:")

for key in peft_model_results:
    improvement = peft_model_results[key] - original_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the PEFT model over the original model:
rouge1: 16.48%
rouge2: 4.48%
rougeL: 9.87%
rougeLsum: 9.76%


Now calculate the improvement of PEFT over a full fine-tuned model:

In [34]:
print("Absolute percentage improvement of the PEFT model over the instruct model:")

for key in peft_model_results:
    improvement = peft_model_results[key] - instruct_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the PEFT model over the instruct model:
rouge1: 4.54%
rouge2: 5.95%
rougeL: 4.68%
rougeLsum: 5.04%


You can see a small percentage decrease in the ROUGE metrics vs. full fine-tuned. However, the training requires much less computing and memory resources.

## Compare Rank 8, 16, 32 PEFT Model

In [42]:
# Evaluate peft model r==8,16
peft_model8 = AutoModelForSeq2SeqLM.from_pretrained('./flan-t5-base-dialogsum-lora8')
peft_model16 = AutoModelForSeq2SeqLM.from_pretrained('./flan-t5-base-dialogsum-lora16')

peft_model8_ouputs = peft_model8.generate(inputs['input_ids'], max_new_tokens=50)
peft_model8_summaries = tokenizer.batch_decode(peft_model8_ouputs, skip_special_tokens=True)

peft_model16_ouputs = peft_model16.generate(inputs['input_ids'], max_new_tokens=50)
peft_model16_summaries = tokenizer.batch_decode(peft_model16_ouputs, skip_special_tokens=True)

In [47]:
peft_model8_results = rouge.compute(
    predictions=peft_model8_summaries,
    references=human_baseline_summaries[0:len(peft_model8_summaries)],
)

peft_model16_results = rouge.compute(
    predictions=peft_model16_summaries,
    references=human_baseline_summaries[0:len(peft_model16_summaries)],
)
print('PEFT MODEL r=8:')
print(peft_model8_results)
print('PEFT MODEL r=16:')
print(peft_model16_results)
print('PEFT MODEL r=32:')
print(peft_model_results)

PEFT MODEL r=8:
{'rouge1': 0.39748221530582584, 'rouge2': 0.09672012236688048, 'rougeL': 0.2870314104500151, 'rougeLsum': 0.28825204975027596}
PEFT MODEL r=16:
{'rouge1': 0.3919381762807263, 'rouge2': 0.11764049230901245, 'rougeL': 0.3043707272880587, 'rougeLsum': 0.3069445630183514}
PEFT MODEL r=32:
{'rouge1': 0.41212303886380713, 'rouge2': 0.13009564455800393, 'rougeL': 0.3103388518865875, 'rougeLsum': 0.3104840695819888}


### Compare PEFT r == 32 with PEFT r == 16

In [48]:
print("Absolute percentage improvement of the PEFT32 model over the PEFT16 model:")

for key in peft_model_results:
    improvement = peft_model_results[key] - peft_model16_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the PEFT32 model over the PEFT16 model:
rouge1: 2.02%
rouge2: 1.25%
rougeL: 0.60%
rougeLsum: 0.35%


### Compare PEFT r == 16 with PEFT r == 8

In [49]:
print("Absolute percentage improvement of the PEFT16 model over the PEFT8 model:")

for key in peft_model_results:
    improvement = peft_model16_results[key] - peft_model8_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the PEFT16 model over the PEFT8 model:
rouge1: -0.55%
rouge2: 2.09%
rougeL: 1.73%
rougeLsum: 1.87%


### Compare PEFT r == 32 with PEFT r == 8

In [50]:
print("Absolute percentage improvement of the PEFT32 model over the PEFT8 model:")

for key in peft_model_results:
    improvement = peft_model_results[key] - peft_model8_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the PEFT32 model over the PEFT8 model:
rouge1: 1.46%
rouge2: 3.34%
rougeL: 2.33%
rougeLsum: 2.22%
